In [1]:
import jax
import projects.charformer.data as data
from importlib import reload
data = reload(data)
import sys
import numpy
import jax
import jax.numpy as jnp
numpy.set_printoptions(threshold=sys.maxsize)
# %pip install tqdm


2024-09-02 05:20:57.782452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 05:20:57.840155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 05:20:57.857914: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-02 05:20:59.653952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [3]:
ds = data.CharDataset(data.CharDataset.get_default_config())

In [4]:
# !  wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt

In [5]:
# ds.create_packed_records(input_file_path='projects/charformer/data/TinyStoriesV2-GPT4-train.txt', output_dir='projects/charformer/data/tfrecords/', custom_delimiter='<|endoftext|>')

In [6]:
retokenized = ds.load_and_retokenize_tfrecord(file_path='projects/charformer/data/tfrecords/record_400.tfrecord')

In [7]:
for r in retokenized:
    print(r)


Once there was a pale piano. It was very shiny and looked like it had just been polished. Bob was a 3 year old boy who loved the piano. Every day, he would come and polish it. He always used his special color polishing cloth. Making the piano shiny and prettier was so much fun for him. 
One day, Bob was very excited. He had some new, sparkly polish that he wanted to use. He was so happy that he was dancing around the piano as he polished it. Suddenly, he knocked off one of the keys and it fell to the ground. Bob was so sad. He didn't know what to do. 
Luckily, his dad was there. He quickly fixed the piano and made it look as good as new. When he was finished, Bob was so proud. The piano was so pale and shiny. He gave his dad a big hug and thanked him for making the piano look so beautiful. From that day on, Bob always remembered to be very careful when polishing the piano.
<|endoftext|


Bella and Max are two cats who live in a big house with a nice family. They like to play together,

In [8]:
iter = ds.create_iterator('projects/charformer/data/tfrecords/record_*.tfrecord', batch_size=8)

In [9]:
batch = next(iter)

In [10]:
import minformer.model as model
import functools
model = reload(model)
data = reload(data)

In [11]:
cfg = model.Config(
    d_model=256,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=4,
    key_dim=128,
    vocab_size=256,
    max_seq_len=8192,
    causal=True,
    use_attn_kernel=True,
    weight_dtype=jnp.float32,
    rules=model.fsdp_rules,
    mesh=model.create_mesh()
)

inference_config = model.Config(
    d_model=256,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=4,
    key_dim=128,
    vocab_size=256,
    max_seq_len=8192,
    causal=True,
    use_attn_kernel=False,
    weight_dtype=jnp.float32,
    rules=model.mdl_parallel_rules,
    mesh=model.create_mesh()
)

In [ ]:
inference_config = model.Config(
    d_model=1024,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=8,
    key_dim=128,
    vocab_size=256,
    max_seq_len=8192,
    causal=True,
    use_attn_kernel=False,
    weight_dtype=jnp.float32,
    rules=model.mdl_parallel_rules,
    mesh=model.create_mesh()
)

# Initialize weights and cache
key = jax.random.PRNGKey(0)
weights = model.Weights.init(inference_config, key, inference_config.mesh, model.mdl_parallel_rules)

In [15]:
import numpy as np
prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
# Define test input sequences
chunk_a = jnp.array([1, 2, 3, 4, 5, 6])
chunk_b = jnp.array([7, 8, 9])
chunk_c = jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

# Prepare chunks
chunk_a, segment_ids_a = model.prepare_chunk(chunk_a, pad_to=16, pad_id=0)
chunk_b, segment_ids_b = model.prepare_chunk(chunk_b, pad_to=16, pad_id=0)
chunk_c, segment_ids_c = model.prepare_chunk(chunk_c, pad_to=16, pad_id=0)

# Run incremental prefill
logits_c, prefill_cache_c = model.forward(chunk_c, segment_ids_c, weights, inference_config, prefill_cache)

prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits_a, prefill_cache = model.forward(chunk_a, segment_ids_a, weights, inference_config, prefill_cache)
logits_b, prefill_cache = model.forward(chunk_b, segment_ids_b, weights, inference_config, prefill_cache)

# Assert cache lengths
assert jnp.array_equal(prefill_cache_c.lengths, jnp.array([9])), "Cache length mismatch for chunk_c"
assert jnp.array_equal(prefill_cache.lengths, jnp.array([9])), "Cache length mismatch for incremental chunks"

# Assert logits shapes
assert logits_c.shape == logits_a.shape == (1, 16, inference_config.vocab_size), "Logits shape mismatch"

# Assert cache consistency
for layer in range(inference_config.num_layers):
    assert np.testing.assert_allclose(prefill_cache_c.k[layer][:, :, :9, :], prefill_cache.k[layer][:, :, :9, :], rtol=1e-4), f"K cache mismatch at layer {layer}"
    assert np.testing.assert_allclose(prefill_cache_c.v[layer][:, :, :9, :], prefill_cache.v[layer][:, :, :9, :], rtol=1e-4), f"V cache mismatch at layer {layer}"

# Assert logits consistency for the first 6 tokens
assert jnp.array_equal(jnp.argmax(logits_c[0, :6], axis=-1), jnp.argmax(logits_a[0, :6], axis=-1)), "Logits mismatch for first 6 tokens"
assert jnp.array_equal(jnp.argmax(logits_c[0, 6:9], axis=-1), jnp.argmax(logits_b[0, :3], axis=-1)), "Logits mismatch for tokens 6 to 9"

print("Incremental prefill test passed!")

DTypePromotionError: The DTypes <class 'numpy.dtypes.Float16DType'> and <class 'numpy.dtype[bfloat16]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

Array([[[[-3.17188, 1.22656, 1.69531, 0.267578, 0.902344, 1.25,
          -1.60156, 0.992188, -1.24219, -0.859375, -1.19531, -0.945312,
          1.8125, 0.378906, -0.90625, 2.07812, -0.414062, 1.05469,
          -2.125, -1.4375, 2.67188, -0.714844, 4.5, 1.9375, -2.71875,
          3.125, -0.423828, 0.147461, -2.03125, 0.695312, 0.675781,
          2.5, 2.4375, -0.46875, 2.03125, 1.08594, -0.333984,
          -0.106934, 2.9375, -0.457031, -0.878906, 1.38281, -0.847656,
          -0.337891, -0.0115967, 1.57031, 0.960938, 0.457031, -2.21875,
          3.3125, -1.41406, -0.882812, 0.224609, -2.09375, 1.76562,
          -1.09375, -0.722656, -1.26562, 2.07812, -1.71094, -0.245117,
          0.835938, 1.27344, -2.20312, -2.28125, 1.76562, 0.478516,
          1.8125, 1.36719, 0.19043, -0.347656, -0.519531, 0.523438,
          0.636719, -1.35938, -0.345703, 1, 0.498047, -1.30469,
          0.242188, 0.257812, 0.131836, 0.204102, -1.67188, -0.609375,
          -0.283203, -0.457031, -0.265625, 1

In [392]:
weights = model.Weights.init(cfg, jax.random.PRNGKey(0), cfg.mesh, model.fsdp_rules)

In [296]:
# jax.tree.map(lambda w: print(w.shape, w.dtype, jnp.mean(w)), weights)

In [393]:
step = jax.jit(model.compute_loss_and_grads, static_argnames='cfg')
step = functools.partial(step, cfg=cfg)

In [394]:
test_batch = jnp.arange(1, 256+2)[None, :]
test_batch = jnp.repeat(test_batch, repeats = 8, axis=0)
batch = {
    'x': test_batch[:, :-1],
    'y': test_batch[:, 1:],
    'segment_ids': jnp.ones((8, 256)),
}

In [395]:
loss, grads = step(weights, batch['x'], batch['segment_ids'], batch['y'])

In [398]:
# Does loss go down!
for i in range(0, 10000):
    # batch = next(iter)
    loss, grads = step(weights, batch['x'], batch['segment_ids'], batch['y'])
    weights = model.update_weights(weights, grads)
    print(loss)

23.835041
18.713705
15.008402
12.28195
10.034771
8.134619
6.532592
5.1551957
3.955792
2.9688058
2.1771111
1.5747759
1.1368322
0.8220417
0.5981845
0.44136527
0.3291421
0.25133616
0.19466925
0.15409064
0.12402634
0.102423236
0.08532847
0.072809756
0.064150415
0.057648856
0.05238415
0.048324905
0.045079987
0.04224724
0.03971554
0.037523758
0.035691604
0.0338479
0.032346774
0.0309695
0.029578695
0.02833714
0.027207453
0.02630578
0.025256682
0.024544002
0.023716252
0.022901395
0.022196598
0.021563455
0.021033928
0.020299222
0.019849114
0.019343792
0.018836876
0.018356416
0.017948668
0.017456807
0.017063418
0.016734531
0.016289964
0.01597264
0.01565715
0.015253962
0.015000738
0.014634695
0.014415642
0.014113752
0.013877286
0.0135574695
0.013354804
0.013102814
0.012884203
0.012675483
0.012465176
0.012225928
0.012100352
0.011911564
0.011732129
0.011505229
0.011323312
0.011185676
0.011007933
0.010826185
0.010712969
0.010547783
0.010382226
0.010261105
0.010100245
0.009980679
0.009866961
0.009740

KeyboardInterrupt: 

In [399]:
logits = model.forward(batch['x'], batch['segment_ids'], weights, cfg)
loss_mask = jnp.where(batch['segment_ids'] == 0, 0, 1)
loss = model.cross_entropy_loss(logits, batch['y'], loss_mask)
loss, jnp.argmax(logits[0, :, :], -1)

(Array(0.00365288, dtype=float32),
 Array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
         41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
         54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
         67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
        145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
        158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
        171, 172

In [400]:
cache = model.KVCache.init(cfg=inference_config, batch_size=8, max_seq_len=2048)
logits, cache = model.forward(batch['x'], batch['segment_ids'], weights, inference_config, cache)
jnp.argmax(logits[0, :, :], -1)

Array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [409]:
prompt = jnp.arange(1, 257)
prompt, prompt_segment_ids = model.prepare_chunk(prompt, pad_to=256, pad_id=0)
cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits, cache = model.forward(prompt, prompt_segment_ids, weights, inference_config, cache)

In [417]:
jnp.argmax(logits[0, :, :], -1)

Array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [420]:
logits[0, 0, :]

Array([  2.2423806 ,   9.842323  ,  26.118044  ,   8.880711  ,
         0.50350976, -11.4651165 ,   8.257105  ,  16.422009  ,
        -0.7391715 ,   6.329277  ,   7.941619  ,  -8.33959   ,
        -3.3781948 ,  16.070179  ,  11.938711  ,  -0.3599987 ,
         0.57523715, -16.514284  ,  -5.1468563 ,  13.056112  ,
        -1.8678036 ,  -7.3458695 ,   1.4935687 , -13.420589  ,
         6.7751656 ,   0.21777022,  -2.6460571 ,   9.581627  ,
        14.255734  ,  11.576826  ,   8.175053  ,  -3.9651523 ,
        17.7677    ,  -1.1233683 ,   2.1667643 ,  -7.667032  ,
         5.943354  ,  -8.832393  ,  -2.7420125 ,   0.97440124,
         9.960157  ,  12.434343  , -12.226874  ,   5.9630866 ,
         5.990312  ,   0.41915607,  12.156579  ,  -3.6476805 ,
         5.0331273 ,  -5.9374566 ,   7.1692533 , -13.306875  ,
       -16.43583   ,  -4.694595  ,  -7.1646466 ,   6.10843   ,
         0.31310463,   1.2385097 ,  -4.559745  ,  -8.294825  ,
         2.0583715 ,  -3.9306364 ,  -3.1013544 ,  -3.65

In [416]:
sampled_tokens = model.sample_next_token(logits, temperature=10)
sampled_tokens

Array([[ 59,  86,  85, 195,  54, 241,   9,  49, 137, 228, 143,  59, 218,
        239,  59, 159,  44, 218,  71, 176, 196, 192, 165,  53,  22, 250,
        130,  80, 174,   9, 199, 119,  98,  48,  28,  66, 237,  75,  39,
        205,  17,  95, 110, 168, 184,  22,  71, 214, 248,  72, 148, 141,
        239,  71, 224, 109, 118,  67, 240, 255, 248, 207, 242, 255, 207,
        180, 236,  80,   5, 182, 147, 172, 164, 126, 184,  64, 243,   4,
        238, 216,  35,   4, 246, 130,   7, 193, 145,   7, 172, 193, 187,
         52, 232, 116,  16, 234,  57, 114, 158, 132,  48, 123, 236, 151,
          2, 245,  19,  70, 154,  59,  78, 115, 212,  60,  82,  29,  18,
        214, 163,  48,  42, 192, 113, 163,  36, 163,  82,  30,  39,  45,
        128, 204, 252, 223, 239, 164, 107,   2,  47, 255,  10,  41, 114,
        196,  46, 166, 173, 222, 116,  79, 149,   3,  54, 227, 225,  43,
         12, 237, 114, 100, 160,  31,  20, 182,  29, 104,  18,  96, 250,
        122, 218,  96, 248, 173,  21,  41,  58, 150

In [344]:
ds.detokenize(batch['x'][0, :])

'<unk>0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c"", \'–\', \'—\', é…\xa0ñà´<oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob><oob>'

In [345]:
# prompt = ds.tokenize('Once upon a time, there was a little cat named Tim.')


In [346]:
logits, cache = model.forward(batch['x'], batch['segment_ids'], weights, inference_config, cache)

In [374]:
jnp.argmax(logits[0, :, :], -1)

Array([177,   2, 121,   4, 178,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  29,  2

In [330]:
sampled_tokens = model.sample_next_token(logits)
sampled_tokens

Array([[ 59,  86,  85, 195,  11, 241,   9,  49, 137, 228, 143,  59, 218,
        239,  59, 159,  44, 218,  71, 176, 196, 192, 165,  53,  22, 250,
        130,  80, 174,   9, 199, 119,  98,  48,  28,  66, 237,  75,  39,
        205,  17,  95, 110, 168, 184,  22,  71, 214, 248,  72, 148, 141,
        239,  71, 224, 109, 118,  67, 240, 255, 248, 207, 242, 255, 207,
        180, 236,  80,   5, 182, 147, 172, 164, 126, 177,  64, 243,   4,
        238, 216,  35,   4, 246, 130,   7, 193, 145,   7, 172, 193, 187,
         52, 232, 116,  16, 234,  57, 114, 158, 132,  48, 123, 236, 151,
          2, 245,  19,  70, 154,  59,  78, 115, 212,  60,  82,  29,  18,
        214, 163,  48,  42, 192, 113, 163,  36, 163,  82,  30,  39,  45,
        128, 204, 252, 223, 239, 164, 107,   2,  47, 255,  10,  41, 114,
        196,  46, 166, 173, 222, 116,  79, 149,   3,  54, 227, 225,  43,
         12, 237, 114, 100, 160,  31,  20, 182,  29, 104,  18,  96, 250,
        122, 218,  96, 248, 173,  21,  41,  58, 150

In [263]:
ds.detokenize(sampled_tokens[0, :])

'W\\[<oob>a<oob>8M<oob><oob><oob>W<oob><oob>W<oob>H<oob>)<oob><oob><oob><oob>Ql<oob><oob><<oob>8<oob><oob>\rLr$<oob>-C<oob>g <oob><oob><oob>l)<oob><oob>*<oob><oob><oob>)<oob>´<oob>%<oob><oob><oob><oob><oob><oob><oob><oob><oob><4<oob><oob><oob><oob><oob><oob>"<oob>3<oob><oob>y3<oob><oob>6<oob><oob>6<oob><oob><oob>P<oob><oob>f<oob>U<oob><oob><oob>L<oob><oob><oob>1<oob>i(<oob>W:<oob><oob>X>sh<oob><oob>LF<oob><oob><oob>z<oob>>tCI<oob><oob><oob><oob><oob><oob>ñ1K<oob>9E<oob><oob>J<oob><oob><oob><oob>;<oob>2R<oob><oob>Gb<oob><oob>\x0c<oob>uj<oob>séh\t<oob><oob><oob>\t<oob><oob>kEV<oob><oob><oob><oob><oob><oob>*<oob><oob><oob><oob>$<oob>=JW\t<oob><oob><oob>, \'–\', \'—\', !<oob>´7<oob><oob>J<oob>\xa03<qT<oob><oob><oob>$$h<oob><oob>E<oob><oob>1<oob><oob>cq<oob><oob><oob><oob><oob>#<oob>t<oob>\r<oob><oob>dh<oob>|~7E<oob><oob><oob>"^)r<oob><oob><oob>'

In [405]:
ds.detokenize(batch['x'][0, :])

'One day, a nice girl named Lucy went to school. She wore a uniform. It was a pretty dress with a blue bow.\nAt school, Lucy saw her friend Tom. Tom had a hole in his uniform. "Oh no, Tom! Your uniform has a hole," she said. Tom was sad. He did not know what to do.\nLucy had an idea. She went home and asked her mom to help. They found a new uniform for Tom. They replaced the old one with the new one. Tom was happy. "Thank you, Lucy!" he said. They played together and had a great day at school.\n<|endoftext|>Once upon a time, in a small town, there was a little girl named Lucy. She loved to eat pastry. One day, she saw a pastry shop and went inside to buy her favorite treat. The shop was full of yummy pastries, and Lucy was very happy.\nLucy picked a pastry that was very hard. She didn\'t know why it was hard, but she wanted to try it. She took a bite, and suddenly, something unexpected happened. The hard pastry turned soft and sweet in her mouth. Lucy was very surprised and happy.\nThe

In [360]:
''.join([ds.itos[int(token)] for token in sampled_tokens[0, 1:3]])

'bv'

In [306]:
chunk_a = jnp.array([1, 2, 3, 4, 5, 6])
chunk_b = jnp.array([7, 8, 9])
chunk_c = jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9])




chunk_a, segment_ids_a = prepare_chunk(chunk_a, pad_to=16)
chunk_b, segment_ids_b = prepare_chunk(chunk_b, pad_to=16)
chunk_c, segment_ids_c = prepare_chunk(chunk_c, pad_to=16)

In [307]:
prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits, prefill_cache = model.forward(chunk_c, segment_ids_c, weights, inference_config, prefill_cache)

In [308]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199,
        0.0688477, 0.0678711, 0.120605, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=bfloat16),
 Array([[25, 25, 25, 25, 25, 67, 67, 67, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

In [309]:
prefill_cache = model.KVCache.init(cfg=inference_config, batch_size=1, max_seq_len=2048)
logits, prefill_cache = model.forward(chunk_a, segment_ids_a, weights, inference_config, prefill_cache)

In [310]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([6], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=bfloat16),
 Array([[25, 25, 25, 25, 25, 67, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

In [311]:
logits, prefill_cache = model.forward(chunk_b, segment_ids_b, weights, inference_config, prefill_cache)

In [312]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.111816, 0.0727539, 0.117188, 0.0407715, 0.12793, 0.0554199,
        0.0688477, 0.0678711, 0.120117, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=bfloat16),
 Array([[67, 67, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]],      dtype=int32))

AssertionError: K cache mismatch at layer 1

In [346]:
jnp.mean(logits_c, axis=-1)

Array([[1.2732564, 1.37221  , 1.4742929, 1.4115022, 1.225872 , 1.3535681,
        1.3480004, 1.2813404, 1.4418206, 1.1841507, 1.1841507, 1.1841507,
        1.1841507, 1.1841507, 1.1841507, 1.1841507]], dtype=float32)

In [345]:
jnp.mean(logits_b, axis=-1)

Array([[1.3480004, 1.2813404, 1.4418232, 1.1840515, 1.1840515, 1.1840515,
        1.1840515, 1.1840515, 1.1841507, 1.1841507, 1.1841507, 1.1841507,
        1.1841507, 1.1841507, 1.1841507, 1.1841507]], dtype=float32)

In [339]:
prefill_cache.lengths, jnp.mean(prefill_cache.k[1], axis=-1)[0, 0, :32], jnp.argmax(logits, axis=-1)

(Array([9], dtype=int32),
 Array([0.194336, 0.232422, 0.160156, 0.25, 0.283203, 0.235352, 0.228516,
        0.1875, 0.237305, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=bfloat16))

In [340]:
prefill_cache_c.lengths, jnp.mean(prefill_cache_c.k[3], axis=-1)[0, 0, :32]

(Array([9], dtype=int32),
 Array([0.194336, 0.232422, 0.160156, 0.25, 0.283203, 0.235352, 0.228516,
        0.1875, 0.237305, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0], dtype=bfloat16))

In [145]:
q_segment_ids = jnp.array([[1,1,2,2], [1,1,1,2]])
k_segment_ids = jnp.array([[1,1,1,2,2,0,0], [1,1,2,2,2,0,0]])

In [151]:
(k_segment_ids[:, None, :] == q_segment_ids[:, :, None]).astype(jnp.int32)

Array([[[1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0]],

       [[1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0]]], dtype=int32)

In [156]:
jnp.tril(jnp.ones((3, 7)))

Array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0.]], dtype=float32)

In [16]:
jax.experimental.shard_map

<module 'jax.experimental.shard_map' from '/home/sholto/.local/lib/python3.10/site-packages/jax/experimental/shard_map.py'>

In [157]:
qk = (1, 1, 3, 7)
q_iota = jax.lax.broadcasted_iota(jnp.int32, qk, 2)
q_offset = 
k_iota = jax.lax.broadcasted_iota(jnp.int32, qk, 3)
mask = q_iota >= k_iota

In [162]:
q_iota.shape

(1, 1, 3, 7)

In [160]:
q_iota

Array([[[[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2, 2]]]], dtype=int32)

In [115]:
model = reload(model)

In [195]:
# Test case 1: Perfect prediction
logits1 = jnp.array([[10.0, 0.0, 0.0],
                        [0.0, 10.0, 0.0]])
labels1 = jnp.array([0, 1])
mask1 = jnp.array([1, 1])

loss1 = model.cross_entropy_loss(logits1, labels1, mask1)
expected_loss1 = 0.0  # Perfect prediction should result in zero loss

print("Test case 1 (Perfect prediction):")
print(f"Logits:\n{logits1}")
print(f"Labels: {labels1}")
print(f"Mask: {mask1}")
print(f"Calculated loss: {loss1:.6f}")
print(f"Expected loss: {expected_loss1:.6f}")
print(f"Test {'passed' if np.isclose(loss1, expected_loss1) else 'failed'}\n")

# Test case 2: Uncertain prediction
logits2 = jnp.array([[1.0, 1.0, 1.0],
                        [1.0, 1.0, 1.0]])
labels2 = jnp.array([0, 2])
mask2 = jnp.array([1, 1])

loss2 = model.cross_entropy_loss(logits2, labels2, mask2)
expected_loss2 = np.log(3)  # log(3) ≈ 1.0986 for uniform distribution over 3 classes

print("Test case 2 (Uncertain prediction):")
print(f"Logits:\n{logits2}")
print(f"Labels: {labels2}")
print(f"Mask: {mask2}")
print(f"Calculated loss: {loss2:.6f}")
print(f"Expected loss: {expected_loss2:.6f}")
print(f"Test {'passed' if np.isclose(loss2, expected_loss2) else 'failed'}\n")

# Test case 3: Mixed predictions with masking
logits3 = jnp.array([[2.0, 1.0, 0.0],
                        [0.0, 2.0, 1.0],
                        [1.0, 0.0, 2.0]])
labels3 = jnp.array([0, 1, 2])
mask3 = jnp.array([1, 1, 0])  # Last prediction is masked

loss3 = model.cross_entropy_loss(logits3, labels3, mask3)

# Manual calculation:
# For [2.0, 1.0, 0.0] and label 0: -log(e^2 / (e^2 + e^1 + e^0)) ≈ 0.4076
# For [0.0, 2.0, 1.0] and label 1: -log(e^2 / (e^0 + e^2 + e^1)) ≈ 0.4076
# Average of these two: (0.4076 + 0.4076) / 2 ≈ 0.4076
expected_loss3 = 0.4076

print("Test case 3 (Mixed predictions with masking):")
print(f"Logits:\n{logits3}")
print(f"Labels: {labels3}")
print(f"Mask: {mask3}")
print(f"Calculated loss: {loss3:.6f}")
print(f"Expected loss: {expected_loss3:.6f}")
print(f"Test {'passed' if np.isclose(loss3, expected_loss3, atol=1e-4) else 'failed'}\n")

Test case 1 (Perfect prediction):
Logits:
[[10.  0.  0.]
 [ 0. 10.  0.]]
Labels: [0 1]
Mask: [1 1]
Calculated loss: 0.000091
Expected loss: 0.000000
Test failed

Test case 2 (Uncertain prediction):
Logits:
[[1. 1. 1.]
 [1. 1. 1.]]
Labels: [0 2]
Mask: [1 1]
Calculated loss: 1.098675
Expected loss: 1.098612
Test failed

Test case 3 (Mixed predictions with masking):
Logits:
[[2. 1. 0.]
 [0. 2. 1.]
 [1. 0. 2.]]
Labels: [0 1 2]
Mask: [1 1 0]
Calculated loss: 0.407657
Expected loss: 0.407600
Test passed



In [187]:
logit = jnp.array([2.0, 1.0, 0.0])
label = jnp.array([0])
label_one_hot = jax.nn.one_hot(label, 3)
print(label_one_hot)
log_softmax = jax.nn.log_softmax(logit)
print(log_softmax)
cross_entropy = label_one_hot * log_softmax
print(cross_entropy)
print(-cross_entropy.sum())

[[1. 0. 0.]]
[-0.40765747 -1.4076575  -2.4076574 ]
[[-0.40765747 -0.         -0.        ]]
0.40765747


In [193]:
model = reload(model)

In [194]:
# Test case 3: Mixed predictions with masking
logits3 = jnp.array([[2.0, 1.0, 0.0],
                        [0.0, 2.0, 1.0],
                        [1.0, 0.0, 2.0]])
labels3 = jnp.array([0, 1, 2])
mask3 = jnp.array([1, 1, 0])  # Last prediction is masked

loss3 = model.cross_entropy_loss(logits3, labels3, mask3)

# Manual calculation:
# For [2.0, 1.0, 0.0] and label 0: -log(e^2 / (e^2 + e^1 + e^0)) ≈ 0.4076
# For [0.0, 2.0, 1.0] and label 1: -log(e^2 / (e^0 + e^2 + e^1)) ≈ 0.4076
# Average of these two: (0.4076 + 0.4076) / 2 ≈ 0.4076
expected_loss3 = 0.4076

print("Test case 3 (Mixed predictions with masking):")
print(f"Logits:\n{logits3}")
print(f"Labels: {labels3}")
print(f"Mask: {mask3}")
print(f"Calculated loss: {loss3}")
print(f"Expected loss: {expected_loss3:.6f}")
print(f"Test {'passed' if np.isclose(loss3, expected_loss3, atol=1e-4) else 'failed'}\n")

Test case 3 (Mixed predictions with masking):
Logits:
[[2. 1. 0.]
 [0. 2. 1.]
 [1. 0. 2.]]
Labels: [0 1 2]
Mask: [1 1 0]
Calculated loss: 0.4076574742794037
Expected loss: 0.407600
Test passed



In [188]:
# Test case 3: Mixed predictions with masking
logits3 = jnp.array([[2.0, 1.0, 0.0],
                        [0.0, 2.0, 1.0]])
labels3 = jnp.array([0, 1])
mask3 = jnp.array([1, 1])  # Last prediction is masked

loss3 = model.cross_entropy_loss(logits3, labels3, mask3)

# Manual calculation:
# For [2.0, 1.0, 0.0] and label 0: -log(e^2 / (e^2 + e^1 + e^0)) ≈ 0.4076
# For [0.0, 2.0, 1.0] and label 1: -log(e^2 / (e^0 + e^2 + e^1)) ≈ 0.4076
# Average of these two: (0.4076 + 0.4076) / 2 ≈ 0.4076
expected_loss3 = 0.4076

print("Test case 3 (Mixed predictions with masking):")
print(f"Logits:\n{logits3}")
print(f"Labels: {labels3}")
print(f"Mask: {mask3}")
print(f"Calculated loss: {loss3:.6f}")
print(f"Expected loss: {expected_loss3:.6f}")
print(f"Test {'passed' if np.isclose(loss3, expected_loss3, atol=1e-4) else 'failed'}\n")

Test case 3 (Mixed predictions with masking):
Logits:
[[2. 1. 0.]
 [0. 2. 1.]]
Labels: [0 1]
Mask: [1 1]
Calculated loss: 0.407657
Expected loss: 0.407600
Test passed



In [119]:
output_attention_kernel[0, 0].mean(axis=-1)

Array([-0.174805, -0.192383, -0.0218506, -0.0285645, -0.0405273,
       -0.0544434, -0.173828, -0.158203, 0.0177002, -0.139648, -0.0186768,
       -0.122559, 0.0791016, 0.0018158, -0.12793, -0.0198975, -0.0498047,
       -0.097168, -0.0549316, -0.0249023, -0.0144653, -0.00585938,
       0.0218506, -0.0668945, -0.024292, -0.00326538, -0.0314941,
       -0.0556641, -0.010437, -0.0239258, -0.00515747, 0.0375977,
       -0.0688477, -0.050293, -0.0583496, -0.0583496, -0.0129395,
       0.000686646, -0.0302734, -0.0771484, -0.00921631, -0.0432129,
       -0.0495605, -0.0213623, -0.0281982, -0.0299072, -0.027832,
       -0.0249023, -0.0712891, -0.0466309, -0.0556641, -0.0368652,
       -0.0294189, -0.0878906, -0.0615234, -0.019043, -0.0449219,
       -0.0332031, -0.074707, -0.0229492, -0.0297852, -0.0262451,
       -0.0566406, -0.0356445, -0.0358887, -0.0153198, -0.0397949,
       -0.0400391, -0.0578613, -0.0368652, -0.0390625, -0.0563965,
       -0.0578613, -0.0262451, 0.00352478, -0.0303955

In [122]:
output_attention[0, 0].mean(axis=-1) - output_attention_kernel[0, 0].mean(axis=-1)

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],      dtype=bfloat16)